In [2]:
"""
resume_bayes_history.py

Flow:
1) Load previous BayesSearchCV cv_results_ (or simulate).
2) Define same skopt.space dimensions as original search.
3) Convert cv_results_['params'] -> list-format aligned to dimensions.
4) Instantiate skopt.Optimizer(n_initial_points=0).
5) opt.tell(...) for every historical point (loss = -score if score is accuracy).
6) Continue loop: opt.ask() -> evaluate via CV -> opt.tell()
7) Refit final estimator on full data with best params found.

Requirements:
    pip install scikit-optimize scikit-learn numpy joblib
"""

from typing import List, Tuple, Dict, Any, Optional
import numpy as np
import pickle, json, os
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from skopt import Optimizer
from skopt.space import Integer, Real, Categorical



In [3]:


# -------------------------
# Helper: mapping utilities
# -------------------------
def dict_to_list_by_dims(pdict: Dict[str, Any], dims: List) -> List[Any]:
    """Convert param dict -> ordered list matching dims order.
       Also apply encode mapping if dim is categorical or sentinel."""
    out = []
    for d in dims:
        name = d.name
        val = pdict.get(name)
        out.append(val)
    return out

def list_to_dict_by_dims(plist: List[Any], dims: List) -> Dict[str, Any]:
    return {d.name: v for d, v in zip(dims, plist)}

def load_cv_results(path: str) -> Dict[str, Any]:
    """Load cv_results from pickle or json."""
    if path.endswith('.pkl') or path.endswith('.pickle'):
        with open(path, 'rb') as f:
            return pickle.load(f)
    elif path.endswith('.json'):
        with open(path, 'r') as f:
            return json.load(f)
    else:
        raise ValueError("Unsupported format. Use .pkl or .json")

# -------------------------
# Example: simulate prior cv_results_ (replace with real load)
# -------------------------
def simulate_prev_cv_results():
    prev = {
        'params': [
            {'n_estimators': 50, 'max_depth': 5, 'min_samples_split': 2, 'criterion': 'gini'},
            {'n_estimators': 100, 'max_depth': 8, 'min_samples_split': 2, 'criterion': 'gini'},
            {'n_estimators': 30, 'max_depth': None, 'min_samples_split': 5, 'criterion': 'entropy'},
            {'n_estimators': 200, 'max_depth': 12, 'min_samples_split': 3, 'criterion': 'gini'},
        ],
        # mean_test_score (higher is better)
        'mean_test_score': [0.82, 0.84, 0.79, 0.85]
    }
    return prev

# -------------------------
# Main resume function
# -------------------------
def resume_bayes_search(cv_results_path: Optional[str] = None,
                        n_new_iter: int = 20,
                        random_state: int = 0):
    """
    cv_results_path: optional path to previous BayesSearchCV.cv_results_ (pickle/json).
                     If None, simulate a previous run.
    n_new_iter: number of new evaluations to run after ingesting history.
    """
    # 1) Define dimensions (MUST match the original search definition exactly)
    dims = [
        Integer(10, 300, name='n_estimators'),
        # For max_depth we prefer using a Categorical that includes None if original allowed None:
        Categorical([None] + list(range(1, 31)), name='max_depth'),
        Integer(2, 10, name='min_samples_split'),
        Categorical(['gini', 'entropy'], name='criterion'),
    ]
    param_names = [d.name for d in dims]
    print("Parameter order:", param_names)

    # 2) Load previous cv_results_
    if cv_results_path is None:
        cv_results = simulate_prev_cv_results()
        print("Using simulated previous cv_results_.")
    else:
        cv_results = load_cv_results(cv_results_path)
        print(f"Loaded previous cv_results_ from {cv_results_path}")

    # Validate presence
    prev_params = cv_results['params']
    prev_scores = np.asarray(cv_results['mean_test_score'], dtype=float)
    assert len(prev_params) == len(prev_scores), "params and mean_test_score length mismatch"

    # 3) Convert prev params dict -> list in dims order
    history: List[Tuple[List[Any], float]] = []
    for p_dict, sc in zip(prev_params, prev_scores):
        plist = dict_to_list_by_dims(p_dict, dims)
        # Skopt expects plain Python types (not masked arrays)
        plist = [int(x) if isinstance(x, (np.integer,)) else x for x in plist]
        history.append((plist, float(sc)))

    print("Extracted history length:", len(history))
    for p, s in history:
        print("  ", p, "->", s)
    print()

    # 4) Instantiate Optimizer - set n_initial_points=0 to avoid extra random init
    opt = Optimizer(
        dimensions=dims,
        base_estimator="GP",
        acq_func="EI",
        random_state=random_state,
        n_initial_points=0
    )

    # 5) Tell optimizer about historical points (skopt minimizes; convert score->loss)
    # If your scores are losses already, don't negate.
    # Here we assume previous scores are accuracies (higher better) -> loss = -score
    for plist, score in history:
        loss = -score
        opt.tell(plist, loss)
    print(f"Told optimizer {len(history)} historical points. Starting optimization...")

    # 6) Prepare evaluation function (CV)
    X, y = make_classification(n_samples=2000, n_features=30, n_informative=10, random_state=0)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

    def evaluate(plist: List[Any]) -> float:
        p_dict = list_to_dict_by_dims(plist, dims)
        # convert types
        n_estimators = int(p_dict['n_estimators'])
        max_depth = p_dict['max_depth']  # possibly None
        if isinstance(max_depth, np.integer):
            max_depth = int(max_depth)
        min_samples_split = int(p_dict['min_samples_split'])
        criterion = p_dict['criterion']
        clf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            criterion=criterion,
            random_state=0,
            n_jobs=-1
        )
        # Use accuracy (higher better)
        scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
        return float(np.mean(scores))

    # 7) Find current best from history
    hist_scores = [s for _, s in history]
    best_score = max(hist_scores)
    best_params_list = history[np.argmax(hist_scores)][0]
    print("Best historical score:", best_score, "params:", best_params_list)
    print()

    # 8) Continue loop: ask -> evaluate -> tell
    for i in range(n_new_iter):
        next_point = opt.ask()
        score = evaluate(next_point)
        opt.tell(next_point, -score)   # pass loss
        if score > best_score:
            best_score = score
            best_params_list = next_point
        print(f"[{i+1:02d}] tried {next_point} -> score {score:.4f} (best {best_score:.4f})")

    # 9) Final best and refit on full data
    best_params = list_to_dict_by_dims(best_params_list, dims)
    # convert types
    best_params['n_estimators'] = int(best_params['n_estimators'])
    if isinstance(best_params['max_depth'], np.integer):
        best_params['max_depth'] = int(best_params['max_depth'])
    best_params['min_samples_split'] = int(best_params['min_samples_split'])
    print("\nFinal best score:", best_score)
    print("Final best params:", best_params)

    final_clf = RandomForestClassifier(random_state=0, n_jobs=-1, **best_params)
    final_clf.fit(X, y)
    print("Refit final_clf on full X,y with best params. Done.")

    # Optionally: save optimizer state or history for later resume
    result_obj = {
        'best_score': best_score,
        'best_params': best_params,
        'optimizer_result': opt.get_result(),  # OptimizeResult
        'history': history
    }
    # Note: opt.get_result() may not be pickle-friendly depending on base_estimator internals.
    return result_obj



In [ ]:
if __name__ == "__main__":
    res = resume_bayes_search(cv_results_path=None, n_new_iter=12)

In [5]:
dims = [
    Integer(10, 300, name='n_estimators'),
    # For max_depth we prefer using a Categorical that includes None if original allowed None:
    Categorical([None] + list(range(1, 31)), name='max_depth'),
    Integer(2, 10, name='min_samples_split'),
    Categorical(['gini', 'entropy'], name='criterion'),
]
param_names = [d.name for d in dims]
print("Parameter order:", param_names)


Parameter order: ['n_estimators', 'max_depth', 'min_samples_split', 'criterion']


In [6]:
dims

[Integer(low=10, high=300, prior='uniform', transform='identity'),
 Categorical(categories=(None, 1, 2, ..., 28, 29, 30), prior=None),
 Integer(low=2, high=10, prior='uniform', transform='identity'),
 Categorical(categories=('gini', 'entropy'), prior=None)]

In [7]:
# Validate presence
prev_params = cv_results['params']
prev_scores = np.asarray(cv_results['mean_test_score'], dtype=float)
assert len(prev_params) == len(prev_scores), "params and mean_test_score length mismatch"


In [8]:
# 3) Convert prev params dict -> list in dims order
history: List[Tuple[List[Any], float]] = []
for p_dict, sc in zip(prev_params, prev_scores):
    plist = dict_to_list_by_dims(p_dict, dims)
    # Skopt expects plain Python types (not masked arrays)
    plist = [int(x) if isinstance(x, (np.integer,)) else x for x in plist]
    history.append((plist, float(sc)))

print("Extracted history length:", len(history))
for p, s in history:
    print("  ", p, "->", s)
print()

Extracted history length: 4
   [50, 5, 2, 'gini'] -> 0.82
   [100, 8, 2, 'gini'] -> 0.84
   [30, None, 5, 'entropy'] -> 0.79
   [200, 12, 3, 'gini'] -> 0.85



In [10]:
n_new_iter: int = 20
random_state: int = 0

# 4) Instantiate Optimizer - set n_initial_points=0 to avoid extra random init
opt = Optimizer(
    dimensions=dims,
    base_estimator="GP",
    acq_func="EI",
    random_state=random_state,
    n_initial_points=0
)

# 5) Tell optimizer about historical points (skopt minimizes; convert score->loss)
# If your scores are losses already, don't negate.
# Here we assume previous scores are accuracies (higher better) -> loss = -score
for plist, score in history:
    loss = -score
    opt.tell(plist, loss)
print(f"Told optimizer {len(history)} historical points. Starting optimization...")


Told optimizer 4 historical points. Starting optimization...


In [11]:
# 6) Prepare evaluation function (CV)
X, y = make_classification(n_samples=2000, n_features=30, n_informative=10, random_state=0)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

def evaluate(plist: List[Any]) -> float:
    p_dict = list_to_dict_by_dims(plist, dims)
    # convert types
    n_estimators = int(p_dict['n_estimators'])
    max_depth = p_dict['max_depth']  # possibly None
    if isinstance(max_depth, np.integer):
        max_depth = int(max_depth)
    min_samples_split = int(p_dict['min_samples_split'])
    criterion = p_dict['criterion']
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        criterion=criterion,
        random_state=0,
        n_jobs=-1
    )
    # Use accuracy (higher better)
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
    return float(np.mean(scores))

# 7) Find current best from history
hist_scores = [s for _, s in history]
best_score = max(hist_scores)
best_params_list = history[np.argmax(hist_scores)][0]
print("Best historical score:", best_score, "params:", best_params_list)
print()

Best historical score: 0.85 params: [200, 12, 3, 'gini']



In [12]:
n_new_iter

20

Lặp n_new_iter vòng: liên tục hỏi optimizer nên thử cấu hình nào (opt.ask()), train + cross-validate để lấy score, báo kết quả lại (opt.tell()).

Tiêu chí chọn: score tốt nhất → update best.

In progress để quan sát live quá trình search.

In [13]:
for i in range(n_new_iter):
    next_point = opt.ask()                # 🧭 Query optimizer cho điểm mới (tham số hyperparam mới)
    score = evaluate(next_point)          # 🧪 Đánh giá điểm (train model, lấy score CV)
    opt.tell(next_point, -score)          # 📩 Ghi lại điểm cho optimizer (chuyển score thành loss, do skopt minimize)
    if score > best_score:                # 🏆 Update nếu tốt hơn best hiện tại
        best_score = score
        best_params_list = next_point
    print(f"[{i+1:02d}] tried {next_point} -> score {score:.4f} (best {best_score:.4f})")


[01] tried [np.int64(300), np.int64(12), np.int64(10), np.str_('gini')] -> score 0.8755 (best 0.8755)
[02] tried [np.int64(300), np.int64(12), np.int64(3), np.str_('entropy')] -> score 0.8735 (best 0.8755)
[03] tried [np.int64(272), np.int64(8), np.int64(5), np.str_('gini')] -> score 0.8690 (best 0.8755)
[04] tried [np.int64(300), np.int64(19), np.int64(8), np.str_('gini')] -> score 0.8800 (best 0.8800)
[05] tried [np.int64(300), np.int64(8), np.int64(10), np.str_('gini')] -> score 0.8645 (best 0.8800)
[06] tried [np.int64(300), np.int64(14), np.int64(7), np.str_('gini')] -> score 0.8785 (best 0.8800)


/Users/ngolong/Documents/CODE_BASED/DevOps-to-MLOps-Bootcamp-Build-Deploy-ML-Systems-End-to-End/venv_mlops/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(300), np.int64(19), np.int64(8), np.str_('gini')] before, using random point [np.int64(290), 18, np.int64(6), 'gini']
  warnings.warn(


[07] tried [np.int64(290), 18, np.int64(6), 'gini'] -> score 0.8820 (best 0.8820)
[08] tried [np.int64(287), np.int64(30), np.int64(10), np.str_('gini')] -> score 0.8775 (best 0.8820)
[09] tried [np.int64(300), np.int64(6), np.int64(10), np.str_('gini')] -> score 0.8415 (best 0.8820)
[10] tried [np.int64(300), np.int64(2), np.int64(9), np.str_('gini')] -> score 0.7625 (best 0.8820)
[11] tried [np.int64(300), np.int64(18), np.int64(9), np.str_('gini')] -> score 0.8785 (best 0.8820)
[12] tried [np.int64(279), np.int64(19), np.int64(9), np.str_('gini')] -> score 0.8785 (best 0.8820)
[13] tried [np.int64(300), np.int64(19), np.int64(2), np.str_('gini')] -> score 0.8775 (best 0.8820)
[14] tried [np.int64(300), np.int64(17), np.int64(9), np.str_('gini')] -> score 0.8780 (best 0.8820)
[15] tried [np.int64(300), np.int64(1), np.int64(4), np.str_('gini')] -> score 0.7435 (best 0.8820)
[16] tried [np.int64(300), np.int64(18), np.int64(10), np.str_('gini')] -> score 0.8755 (best 0.8820)
[17] trie

/Users/ngolong/Documents/CODE_BASED/DevOps-to-MLOps-Bootcamp-Build-Deploy-ML-Systems-End-to-End/venv_mlops/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(300), np.int64(18), np.int64(2), np.str_('gini')] before, using random point [np.int64(219), 2, np.int64(2), 'entropy']
  warnings.warn(


[18] tried [np.int64(219), 2, np.int64(2), 'entropy'] -> score 0.7555 (best 0.8820)


/Users/ngolong/Documents/CODE_BASED/DevOps-to-MLOps-Bootcamp-Build-Deploy-ML-Systems-End-to-End/venv_mlops/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(300), np.int64(18), np.int64(2), np.str_('gini')] before, using random point [np.int64(80), 16, np.int64(4), 'entropy']
  warnings.warn(


[19] tried [np.int64(80), 16, np.int64(4), 'entropy'] -> score 0.8690 (best 0.8820)
[20] tried [np.int64(300), np.int64(16), np.int64(9), np.str_('gini')] -> score 0.8805 (best 0.8820)


In [17]:
# 9) Final best and refit on full data
best_params = list_to_dict_by_dims(best_params_list, dims)

# convert types
best_params['n_estimators'] = int(best_params['n_estimators'])
if isinstance(best_params['max_depth'], np.integer):
    best_params['max_depth'] = int(best_params['max_depth'])
best_params['min_samples_split'] = int(best_params['min_samples_split'])
print("\nFinal best score:", best_score)
print("Final best params:", best_params)

final_clf = RandomForestClassifier(random_state=0, n_jobs=-1, **best_params)
final_clf.fit(X, y)
print("Refit final_clf on full X,y with best params. Done.")

# Optionally: save optimizer state or history for later resume
result_obj = {
    'best_score': best_score,
    'best_params': best_params,
    'optimizer_result': opt.get_result(),  # OptimizeResult
    'history': history
}

print("result" + " " + "-"*100)
print(result_obj)


Final best score: 0.8819981900941422
Final best params: {'n_estimators': 290, 'max_depth': 18, 'min_samples_split': 6, 'criterion': 'gini'}
Refit final_clf on full X,y with best params. Done.
result ----------------------------------------------------------------------------------------------------
{'best_score': 0.8819981900941422, 'best_params': {'n_estimators': 290, 'max_depth': 18, 'min_samples_split': 6, 'criterion': 'gini'}, 'optimizer_result':           fun: -0.8819981900941422
            x: [np.int64(290), 18, np.int64(6), 'gini']
    func_vals: [-8.200e-01 -8.400e-01 ... -8.690e-01 -8.805e-01]
      x_iters: [[50, 5, 2, 'gini'], [100, 8, 2, 'gini'], [30, None, 5, 'entropy'], [200, 12, 3, 'gini'], [np.int64(300), np.int64(12), np.int64(10), np.str_('gini')], [np.int64(300), np.int64(12), np.int64(3), np.str_('entropy')], [np.int64(272), np.int64(8), np.int64(5), np.str_('gini')], [np.int64(300), np.int64(19), np.int64(8), np.str_('gini')], [np.int64(300), np.int64(8), np.int6